# Mapping Table 1

In [39]:
# 라이브러리 호출
import pandas as pd

# 데이터 불러오기
dfp = pd.read_excel("Product_crawl_220803_touched.xlsx")

# 해당부분만 떼서 각 Index 별로 분배
trend_list = []
index_list = []
for i in dfp.columns[27:-2]:
    for l in range(len(dfp)):
        if dfp[i][l] > 0:
            index_list.append(dfp[dfp.columns[0]][l])
            trend_list.append(i)

# 데이터프레임 생성
dfn = pd.DataFrame({'index' :index_list,
                'trend':trend_list})

# 데이터 Export
dfn.sort_values(by='index').to_excel('trend_mapping.xlsx', index=False)


# Mapping Table 2

In [ ]:
# 데이터 불러오기 (사업 / 인구 )
dfw = pd.read_excel("work.xlsx")
dfh = pd.read_excel("human.xlsx")

# 데이터 컬럼 정리
dfh = dfh[['행정구역','2022년06월_계_총인구수']]
dfw = dfw[['시도명','사업명']]

# 데이터 전처리
temp_list = []
for i in dfh['행정구역']:
    temp_list.append(i.split()[0])
dfh['행정구역'] = temp_list

# 시도별, 사업별 개수
temp_list_1 = []
temp_list_2 = []
temp_list_3 = []
for i in dfh['행정구역']: # 행정구역들
    for l in dfw['사업명'].unique(): # 사업명 4개
        temp = (dfw['시도명'] == i) & (dfw['사업명'] == l)
        try:
            temp_list_3.append(temp.value_counts()[1])
        except:
            temp_list_3.append(0)
        temp_list_1.append(i)
        temp_list_2.append(l)

# 데이터프레임으로 생성
df = pd.DataFrame({'시도명':temp_list_1,
                    '사업명':temp_list_2,
                    '사업수':temp_list_3})

# 엑셀로 Export
df.to_excel('trend_mapping_forU.xlsx', index=False)

# EDA Table 1

In [110]:
dfc = pd.read_excel("company_crawl_220803.xlsx")
import numpy as np
temp_list = []
for i in dfc['co_empl']:
    try:
        temp_list.append(i.replace("+","").replace(",","").split()[0])
    except:
        temp_list.append(np.nan)
dfc['co_empl'] = temp_list
dfc.to_excel('bruh_output.xlsx')

# EDA Table 2

In [2]:
# 데이터 불러오기
import pandas as pd
dfp = pd.read_excel("Product_crawl_220803_touched.xlsx")

In [6]:
dfp.columns

Index(['index', 'pd_name', 'pd_link', 'pd_pric', 'pd_srch', 'pd_rank',
       'pd_cat1', 'pd_cat2', 'pd_cat3', 'pd_cat4', 'pd_stoc', 'pd_cond',
       'pd_brnd', 'pd_mpn', 'pd_upc', 'pd_cont', 'pd_modl', 'pd_type',
       'pd_desc', 'desc_len', 'license', 'construct', 'green', 'category',
       'garden', 'tf', 'tp', '친환경 · 에너지 효율  건축지원', '독일 스프링클러 기술표준',
       'VDS(독일 보험협회) 소방인증', 'NFPA 13 설치표준', 'UL 소방인증', '친환경 소화제',
       '물안개 소화 시스템 (Water mist extinguishing systems)', '부식방지 폴리머 강화 파이프',
       '인공지능 소화 시스템', '산소감소 소화 시스템', '준비작동식 스프링클러', '서리 방지 스프링클러 소화제',
       '거품 소화 시스템', '분말 소화 시스템', '랙 장비 소화 시스템', '가스 소화 시스템', '스프링클러 부식 방지 기술',
       '스프링클러 동결 방지 기술', '포탑 소화 시스템', '전자기기용 전구형 소화장치', '재활용 가능한 소방용품',
       '소방호스 업사이클링', 'Trend Fitness', 'Trend Fit Point'],
      dtype='object')

In [11]:
dfv = dfp[['index','pd_desc']]

# Word2Vec

In [17]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [9]:
dfv

,index,pd_desc
0,0,eBayVicflex Dry Sprinkler Style VS1-50? ? ? 50...
1,1,eBayNOS?(2)?Viking Pendent?155F?Brass Fire Spr...
2,2,eBayNOS165*F?Fusible Link?Brass Pendant?Fire S...
3,3,eBayRooftop Sprinkler Embers-Guard™Ember-S-gua...
4,4,eBayLot of 4 Antique Brass Fire Sprinkler Head...
...,...,...
12795,12797,eBayGarden Watering Nozzle Irrigation Lawn Osc...
12796,12798,eBay ...
12797,12799,eBayGarden Watering Nozzle Irrigation Lawn Spr...
12798,12800,eBayProduct Description1PCS Long Distance Gard...


In [4]:
import re
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
stop_word_eng = set(stopwords.words('english'))
ps_stemmer = PorterStemmer()
token = RegexpTokenizer('[\w]+')
tokenized = []

for t in tqdm(dfv['pd_desc']):
    try:
        text = t.lower().split(" ")
        #print(text)
        compile = re.compile("\W+")
        line = [i for i in text if i not in stop_word_eng]
        result = [token.tokenize(i) for i in line]
        middle_result= [r for i in result for r in i]
        result = [ps_stemmer.stem(i) for i in middle_result if not i in stop_word_eng] # 불용어 제거
        #result = [i for i in final_result if i not in stop_word]

        tokenized.append(result)
    except:
        tokenized.append([])

eng = pd.Series(sum(tokenized,[]))

100%|██████████| 12800/12800 [00:52<00:00, 241.78it/s]


In [18]:
result_list = []
for t in tqdm(dfv['pd_desc']):
    try:
        content_text = re.sub(r'\([^)]*\)', '', t)
        sent_text = sent_tokenize(content_text)
        normalized_text = []
        for string in sent_text:
            tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
            normalized_text.append(tokens)
        result = [word_tokenize(sentence) for sentence in normalized_text]
        result_list.append(result)
    except:
        result_list.append([])


100%|██████████| 12800/12800 [00:28<00:00, 450.18it/s]


In [19]:
result=[y for x in result_list for y in x]

In [20]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [82]:
source = []
target = []
views = []

for i in ['energy','regulation','vds','nfpa','ul','eco','mist','coating','oxygen','pre','antifreeze','foam','powder','cabinet','gas','corrosion','freeze','bulb','recycle','up']:
    model_result = model.wv.most_similar(i)
    for l in model_result:
        source.append(i)
        target.append(l[0]) # target
        views.append(l[1]) # Views

import pandas as pd
df = pd.DataFrame({'Source':source,
                    'Target':target,
                    'views':views})

In [83]:
df

,Source,Target,views
0,energy,efficient,0.763556
1,energy,eco,0.653016
2,energy,environmentally,0.635329
3,energy,friendly,0.627999
4,energy,retention,0.625648
...,...,...,...
195,up,subject,0.323470
196,up,longer,0.312627
197,up,carry,0.309187
198,up,changing,0.306124


In [85]:
# 저장
df.to_csv('Network_table.csv', encoding='utf-8')

In [81]:
model_result = model.wv.most_similar("energy")
model_result[0][0]

'efficient'

In [69]:
model_result = model.wv.most_similar("regulation")
print(model_result)

[('phase', 0.589060366153717), ('siliconel', 0.5845210552215576), ('determined', 0.5804451704025269), ('municipal', 0.5757759213447571), ('adjusting', 0.540707528591156), ('handbook', 0.5373601913452148), ('city', 0.5350537300109863), ('administration', 0.5320284962654114), ('michael', 0.5175769925117493), ('david', 0.5102512240409851)]


In [36]:
model_result = model.wv.most_similar("vds")
print(model_result)

[('yessafety', 0.6210481524467468), ('systemzero', 0.6162407994270325), ('ebaymanufacturer', 0.5937943458557129), ('vk102', 0.5925118923187256), ('d1', 0.591480553150177), ('ebay50pc', 0.5909313559532166), ('800w', 0.5905582904815674), ('ex6430', 0.5882773399353027), ('irrigationdescription50x', 0.5831693410873413), ('description50pc', 0.5825844407081604)]


In [71]:
model_result = model.wv.most_similar("nfpa")
print(model_result)

[('publisher', 0.6909229159355164), ('astm', 0.6857380270957947), ('csa', 0.6847530007362366), ('viking', 0.6461664438247681), ('meets', 0.6376650929450989), ('13d', 0.6257234811782837), ('author', 0.6247117519378662), ('waterflow', 0.6219533085823059), ('9001', 0.6167435646057129), ('detector', 0.6149163842201233)]


In [43]:
model_result = model.wv.most_similar("ul")
print(model_result)

[('ulc', 0.8036059141159058), ('fm', 0.7556964159011841), ('cul', 0.7519188523292542), ('astm', 0.7363535165786743), ('ex6430', 0.7317100167274475), ('yesnsf', 0.7307905554771423), ('culus', 0.7157650589942932), ('csa', 0.6980778574943542), ('certified', 0.6942856907844543), ('9001', 0.6852016448974609)]


In [72]:
model_result = model.wv.most_similar("eco")
print(model_result)

[('environmentally', 0.7663635611534119), ('friendly', 0.7596206665039062), ('elegant', 0.7487348914146423), ('abrasion', 0.7038860321044922), ('adds', 0.6557727456092834), ('energy', 0.6530157923698425), ('superior', 0.6436828970909119), ('lightweight', 0.6410384774208069), ('windproof', 0.6394991278648376), ('advantages', 0.6371086835861206)]


In [74]:
model_result = model.wv.most_similar("mist")
print(model_result)

[('particles', 0.5560967922210693), ('fine', 0.5453689694404602), ('inflatable', 0.5375937223434448), ('modes', 0.5331375002861023), ('calibre', 0.5304380059242249), ('bubbler', 0.513094961643219), ('rockwood', 0.5058239698410034), ('jet', 0.5054169297218323), ('emitter', 0.5030114054679871), ('atomization', 0.5016798973083496)]


In [75]:
model_result = model.wv.most_similar("coating")
print(model_result)

[('flue', 0.7372561097145081), ('tobacco', 0.7364110350608826), ('coated', 0.692889928817749), ('truckc', 0.6903624534606934), ('painted', 0.671558678150177), ('gas', 0.6685886383056641), ('decorative', 0.6664612293243408), ('rescue', 0.6617200970649719), ('windproof', 0.6565675139427185), ('role', 0.6548659205436707)]


In [52]:
model_result = model.wv.most_similar("oxygen")
print(model_result)

[('flames', 0.6730307340621948), ('burning', 0.6455993056297302), ('squeeze', 0.5894021987915039), ('sweep', 0.5875570774078369), ('utilities', 0.5582514405250549), ('stop', 0.5442023277282715), ('oil', 0.5430602431297302), ('eliminate', 0.5413143634796143), ('preventing', 0.5408375859260559), ('discharged', 0.537423312664032)]


In [77]:
model_result = model.wv.most_similar("pre")
print(model_result)

[('mounting', 0.5539923906326294), ('escutcheon', 0.5074782371520996), ('bonnet', 0.49280092120170593), ('plain', 0.47502148151397705), ('recessed', 0.46573445200920105), ('bathroom', 0.45775365829467773), ('backing', 0.4534775912761688), ('620', 0.4531228542327881), ('label', 0.45212623476982117), ('nbr', 0.449749231338501)]


In [63]:
model_result = model.wv.most_similar("antifreeze")
print(model_result)

[('examines', 0.5059763193130493), ('spaces', 0.4736999571323395), ('soaker', 0.46562254428863525), ('gardening', 0.46246975660324097), ('garage', 0.451834112405777), ('future', 0.44594019651412964), ('engsynopsis', 0.4443955421447754), ('kit', 0.43375447392463684), ('kettle', 0.4335917532444), ('borders', 0.4303920567035675)]


In [56]:
model_result = model.wv.most_similar("foam")
print(model_result)

[('dioxide', 0.7763537764549255), ('carbon', 0.7174956202507019), ('afff', 0.7119457125663757), ('sodium', 0.6681053638458252), ('aerial', 0.6586971879005432), ('truckc', 0.6484240293502808), ('extinguishant', 0.6349611878395081), ('treatment', 0.6252532601356506), ('lance', 0.6236169338226318), ('powder', 0.6208654642105103)]


In [58]:
model_result = model.wv.most_similar("powder")
print(model_result)

[('dry', 0.7539325952529907), ('abe', 0.7429816722869873), ('truckc', 0.6985610127449036), ('carbon', 0.6854802370071411), ('coated', 0.6808155179023743), ('aerial', 0.6646528244018555), ('1kg', 0.663991391658783), ('ammonium', 0.6627735495567322), ('cylinders', 0.6594997048377991), ('chemical', 0.654915452003479)]


In [59]:
model_result = model.wv.most_similar("cabinet")
print(model_result)

[('round', 0.7631329894065857), ('semi', 0.7395769357681274), ('tempered', 0.7285898923873901), ('lock', 0.721603274345398), ('recessed', 0.718862771987915), ('molded', 0.7126045823097229), ('leather', 0.7092360258102417), ('latch', 0.7051291465759277), ('acrylic', 0.702668309211731), ('foot', 0.6978692412376404)]


In [60]:
model_result = model.wv.most_similar("gas")
print(model_result)

[('coating', 0.6685886383056641), ('aerial', 0.6533563733100891), ('fighting', 0.6304861307144165), ('truckc', 0.6291289329528809), ('cartridge', 0.6250663995742798), ('signaling', 0.6247497797012329), ('oil', 0.6247192621231079), ('wet', 0.6155312657356262), ('suppression', 0.6148420572280884), ('ladder', 0.6124799847602844)]


In [61]:
model_result = model.wv.most_similar("corrosion")
print(model_result)

[('tough', 0.803975522518158), ('resistant', 0.7802555561065674), ('gf', 0.7290877103805542), ('abrasion', 0.7275943160057068), ('lightweight', 0.7199957370758057), ('durability', 0.6908071637153625), ('hightemperature', 0.689023494720459), ('fungal', 0.6889410614967346), ('uv', 0.6858384609222412), ('moisture', 0.6645101308822632)]


In [62]:
model_result = model.wv.most_similar("freeze")
print(model_result)

[('instructional', 0.6945998072624207), ('smoke', 0.6552724838256836), ('serves', 0.6552177667617798), ('dvd', 0.6015099287033081), ('deliveryvisit', 0.5959595441818237), ('upspray', 0.580348789691925), ('bpa', 0.5753657817840576), ('listerthe', 0.5683583617210388), ('instant', 0.5595260858535767), ('labyrinth', 0.5565648674964905)]


In [65]:
model_result = model.wv.most_similar("bulb")
print(model_result)

[('frangible', 0.7180282473564148), ('coated', 0.7003592848777771), ('pendent', 0.6973341703414917), ('strobe', 0.6860014796257019), ('ceiling', 0.6755885481834412), ('upright', 0.672823429107666), ('led', 0.6694995164871216), ('graphic', 0.6694039106369019), ('155f', 0.6665119528770447), ('glass', 0.6652551293373108)]


In [66]:
model_result = model.wv.most_similar("recycle")
print(model_result)

[('specialized', 0.5715609788894653), ('58cm', 0.5152342915534973), ('equipment', 0.4976256191730499), ('fully', 0.4478294551372528), ('corp', 0.4443677067756653), ('bracketsfeature', 0.4319995939731598), ('2xfeature', 0.421805739402771), ('vintage', 0.4148240089416504), ('shops', 0.4140331447124481), ('farm', 0.411601185798645)]


In [68]:
model_result = model.wv.most_similar("up")
print(model_result)

[('piecesnotethis', 0.3499282896518707), ('rated', 0.338899701833725), ('50', 0.3357253074645996), ('news', 0.3298394978046417), ('000', 0.3264506459236145), ('subject', 0.32347041368484497), ('longer', 0.3126266598701477), ('carry', 0.30918678641319275), ('changing', 0.3061237335205078), ('returnsreturnsyou', 0.30592891573905945)]


In [23]:
all = pd.Series(sum(result,[]))

In [33]:
all.value_counts(20)

the                    3.504972e-02
to                     2.538248e-02
and                    2.297415e-02
of                     1.605111e-02
for                    1.316191e-02
                           ...     
manualunderstanding    2.217004e-07
0145                   2.217004e-07
steelrechargeable      2.217004e-07
lbtemperature          2.217004e-07
states40               2.217004e-07
Length: 49004, dtype: float64